# adam_model.ipynb
Description   : Multiple layer perceptron to solve general problems<br>

In [7]:
%run mlp_model.ipynb

아담 알고리즘의 적용 여부에 따른 성능 관찰이 필요하다. 이때 플래그 값(self.use_adam)을 초기화 메서드에서 설정해준다.

In [8]:
class AdamModel(MlpModel):
    def __init__(self, name, dataset, hconfigs): # 초기화 메서드, 이름, 데이터 셋, 은닉층 정보
        self.use_adam = False # 플래그
        super(AdamModel, self).__init__(name, dataset, hconfigs)

In [9]:
# Adam 역전파 신경망
def adam_backprop_layer(self, G_y, hconfig, pm, aux):
    x, y = aux

    if hconfig is not None: G_y = relu_derv(y) * G_y

    g_y_weight = x.transpose()
    g_y_input = pm['w'].transpose()

    G_weight = np.matmul(g_y_weight, G_y)
    G_bias = np.sum(G_y, axis=0)
    G_input = np.matmul(G_y, g_y_input)

    # 기존 역전파 연산에서 달라지는 부분
    self.update_param(pm, 'w', G_weight) # 가중치
    self.update_param(pm, 'b', G_bias) # 편향

    return G_input

AdamModel.backprop_layer = adam_backprop_layer

In [10]:
# parameter 갱신
def adam_update_param(self, pm, key, delta):
    if self.use_adam:
        delta = self.eval_adam_delta(pm, key, delta)
    
    pm[key] -= self.learning_rate * delta
        
AdamModel.update_param = adam_update_param

In [12]:
def adam_eval_adam_delta(self, pm, key, delta):
    # adam 논문에 나온 변수값 목록
    beta_1 = 0.9
    beta_2 = 0.999
    epsilon = 1.0e-8
    
    vkey, gkey, step = 'v' + key, 'g' + key, 's' + key
    # 각 변수 초기화 부분
    if vkey not in pm:
        pm[vkey] = np.zeros(pm[key].shape)
        pm[gkey] = np.zeros(pm[key].shape)
        pm[step] = 0
        
    v = beta_1 * pm[vkey] + (1 - beta_1) * delta
    g = beta_2 * pm[gkey] + (1 - beta_2) * (delta * delta)
    
    pm[step] += 1
    v = v / (1 - np.power(beta_1, pm[step]))
    g = g / (1 - np.power(beta_2, pm[step]))
    
    return v / (np.sqrt(g) + epsilon)

AdamModel.eval_adam_delta = adam_eval_adam_delta